In [28]:
import sys
import re
import os

def find_binary_data(filename,start):    
    f = open(filename, "r")
    f.seek(start)
    count = 0
    for line in f:
        llen = len(line)
        count += llen
        if line.startswith("beginimage"):
            break
    f.close()
    return count

colors = []
data_size = 0
def parse_line(line):
    global colors
    global data_size
    # %%BoundingBox: 0 0 609 779
    # parse bounding box
    match = re.search(r"%%BoundingBox: (\d+) (\d+) (\d+) (\d+)", line)
    if(match):
        print("Bounding box: ", match.group(1), match.group(2), match.group(3), match.group(4))

    # %%CMYKCustomColor: 0.39 0.13 0.01 0.03 (Cyan)
    # parse custom color
    match = re.search(r"%%CMYKCustomColor: (\d+\.\d+) (\d+\.\d+) (\d+\.\d+) (\d+\.\d+) \((\w+)\)", line)
    if(match):
        print("Custom color: ", match.group(1), match.group(2), match.group(3), match.group(4), match.group(5))
        colors.append(match.group(5))
    # %%BeginBinary: 946000
    # parse binary data
    match = re.search(r"%%BeginBinary: (\d+)", line)
    if(match):
        print("Binary data: ", match.group(1))
        data_size = int(match.group(1))


 # Open the eps file
f = open("2.eps", "r")
# Read the file line by line until the begin image tag
line_count = 0
byte_count = 0
for line in f:    
    line_count += 1
    llen = len(line)
    byte_count += llen
    if line.startswith("beginimage"):
        break
    if llen != 0:
        parse_line(line)
print("line count: ", line_count)
print("byte count: ", byte_count)
print("colors_count: ", len(colors))
print("data_size: ", data_size)
# Close the file
f.close()
# Read the file as binary data 
f = open("2.eps", "rb")
f.seek(byte_count)

# Read the binary data
data = []
for color in colors:
    f = open("2.eps", "rb")
    f.seek(byte_count)
    print("color: ", color)    
    d = f.read(data_size)
    data.append(d)
    f.close()
    byte_count += data_size
    pos = find_binary_data("2.eps", byte_count)
    byte_count += pos

# Open the output file
f = open("2.bin", "wb")
# Write the binary data
for d in data:
    f.write(d)
# Close the file
f.close()


Bounding box:  0 0 609 779
Custom color:  0.39 0.13 0.01 0.03 Cyan
Custom color:  0.01 0.39 0.18 0.02 Magenta
Custom color:  0.00 0.01 0.38 0.00 Yellow
Custom color:  0.22 0.22 0.21 0.32 Black
Custom color:  0.00 0.26 0.37 0.00 Orange
Custom color:  0.39 0.24 0.05 0.14 Blue
Bounding box:  0 0 609 779
Binary data:  946000
line count:  110
byte count:  3814
colors_count:  6
data_size:  946000
color:  Cyan
color:  Magenta
color:  Yellow
color:  Black
color:  Orange
color:  Blue


In [ ]:
# Search for the binary data
match = re.search(r"%%BeginData: (\d+) Hex Bytes", data)
# Get the number of bytes
bytes = int(match.group(1))
# Search for the binary data
match = re.search(r"%%BeginData (.*) %%EndData", data, re.DOTALL)
# Get the binary data
binary = match.group(1)
# Remove the spaces
binary = binary.replace(" ", "")
# Remove the new lines
binary = binary.replace("\n", "")
# Convert the binary data to a string
binary = binary.decode("hex")
# Open the output file
f = open(output_file, "wb")
# Write the binary data
f.write(binary)
# Close the file
f.close()